# Aposteriori Unimodality applications

## Real life data

### Preprocessing

In [3]:
%load_ext autoreload 
%autoreload 2
import pandas as pd
import numpy as np
from pathlib import Path

import aposteriori

import re
import hashlib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
df = pd.read_pickle("data/attitudes_embedded.csv")
df

,tweet,toyou,toany,annotatorMinority,bert,annotatorPolitics,traditionalism,annotatorRace,annotatorAge,annotatorGender,freeSpeech,harmHateSpeech,intent,lingPurism,racism,racist
0,""" you ugly bitch!!!""- that seems to be my real...","[2, 1, 1, 2]","[3, 3, 1, 2]","[{}, pansexual, disabled, trans, {}, {}]","[0.0047034588642418385, 0.11112377792596817, 0...","[-1.0, -1.0, 1.0, 0.5]","[3, 1, 5, 4]","[black, black, white, white]","[25.0, 35.0, 50.0, 25.0]","[man, man, woman, woman]","[1, 1, 1, 2]","[4, 5, 5, 2]","[1, 1, 1, 2]","[1, 1, 5, 5]","[1, 1, 1, 4]","[1, 1, 1, 1]"
1,"""""""I am happy to report that drowning does kil...","[4, 2, 1, 2, 1, 1]","[4, 2, 1, 2, 2, 1]","[{}, {}, {}, {}, {}, {}]","[0.35706740617752075, 0.032704465091228485, 0....","[-0.5, -0.5, 0.5, 0.5, -1.0, -1.0]","[4, 3, 4, 3, 1, 4]","[black, black, white, white, white, white]","[40.0, 30.0, 40.0, 30.0, 25.0, 25.0]","[woman, woman, woman, man, man, man]","[1, 1, 3, 4, 1, 1]","[4, 4, 3, 1, 5, 4]","[4, 2, 1, 3, 1, 1]","[2, 5, 5, 5, 5, 5]","[1, 1, 5, 5, 1, 1]","[1, 1, 1, 1, 1, 1]"
2,"""""""Let them praise the name of the LORD: for h...","[1, 1, 1, 1, 1, 2]","[2, 1, 1, 1, 1, 3]","[{}, black; female, nan, {}, {}, {}]","[-0.022276882082223892, 0.17076537013053894, 0...","[-0.5, -1.0, 1.0, 0.0, -1.0, -0.5]","[3, 4, 3, 5, 1, 3]","[black, black, white, white, white, white]","[30.0, 40.0, 35.0, 40.0, 25.0, 45.0]","[woman, woman, man, woman, man, woman]","[1, 1, 5, 1, 1, 1]","[4, 5, 1, 5, 5, 4]","[1, 1, 1, 1, 1, 1]","[5, 4, 5, 5, 5, 4]","[1, 1, 5, 1, 1, 1]","[1, 1, 1, 1, 1, 1]"
3,"""""""That was decades ago, Cecil! Anyway, we wan...","[1, 2, 1, 1, 1, 1]","[1, 4, 1, 1, 1, 1]","[{}, black; female, nan, {}, {}, nan]","[0.07474598288536072, 0.24487990140914917, 0.2...","[0.0, -1.0, 1.0, 0.5, -0.5, -1.0]","[4, 4, 3, 4, 5, 5]","[black, black, white, white, white, white]","[35.0, 40.0, 35.0, 40.0, 35.0, 45.0]","[man, woman, man, woman, woman, woman]","[1, 1, 5, 3, 1, 1]","[4, 5, 1, 3, 5, 5]","[1, 1, 1, 1, 1, 1]","[4, 4, 5, 5, 4, 3]","[1, 1, 5, 5, 1, 1]","[1, 1, 1, 1, 1, 1]"
4,"""""""This Jestxh idiot keeps riding me in dms, t...","[1, 3, 3, 3, 4]","[1, 2, 4, 4, 4]","[{}, NONE, No i do not, none, {}]","[0.050774428993463516, 0.08187347650527954, 0....","[-1.0, 1.0, 0.5, 0.5, -1.0]","[5, 3, 5, 5, 5]","[black, black, white, white, white]","[45.0, 45.0, 45.0, 60.0, 40.0]","[man, man, woman, woman, woman]","[1, 5, 1, 1, 1]","[5, 4, 3, 5, 2]","[1, 3, 1, 3, 5]","[5, 3, 4, 5, 5]","[1, 3, 5, 1, 1]","[1, 4, 2, 2, 1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,women are not dumb bitches,"[1, 5, 2, 3]","[4, 5, 2, 3]","[pansexual, disabled, trans, {}, none, {}]","[0.22457970678806305, 0.3840716481208801, -0.4...","[-1.0, 1.0, 0.5, 0.5]","[1, 5, 5, 5]","[black, black, white, white]","[35.0, 30.0, 45.0, 65.0]","[man, woman, man, woman]","[1, 5, 1, 1]","[5, 5, 5, 5]","[1, 5, 1, 3]","[1, 5, 5, 5]","[1, 5, 1, 1]","[1, 1, 1, 1]"
622,worst part about getting highlights is sitting...,"[1, 3, 1, 1, 1, 1]","[1, 5, 1, 1, 1, 1]","[{}, NONE, {}, {}, {}, none]","[0.1781517118215561, 0.02407737821340561, 0.13...","[-1.0, 1.0, 0.5, 0.5, -0.5, -0.5]","[5, 3, 5, 2, 4, 3]","[black, black, white, white, white, white]","[45.0, 45.0, 35.0, 35.0, 25.0, 50.0]","[man, man, woman, man, man, man]","[1, 5, 1, 2, 1, 1]","[5, 4, 4, 2, 3, 5]","[1, 3, 1, 1, 1, 1]","[5, 3, 5, 2, 5, 4]","[1, 5, 2, 3, 2, 1]","[1, 4, 1, 1, 1, 1]"
623,yids are not as honest as the rest of us,"[5, 5, 4, 5, 4, 1]","[5, 5, 4, 5, 4, 1]","[nan, {}, {}, none, {}, bisexual]","[0.2983386516571045, 0.31099164485931396, -0.4...","[-0.5, -0.5, 0.5, 0.5, -0.5, -0.5]","[3, 5, 2, 5, 4, 3]","[black, black, white, white, white, white]","[30.0, 30.0, 40.0, 60.0, 35.0, 40.0]","[man, man, man, woman, woman, woman]","[1, 1, 5, 1, 2, 1]","[3, 3, 2, 5, 3, 5]","[5, 5, 4, 5, 5, 1]","[5, 3, 5, 5, 4, 5]","[1, 1, 4, 1, 2, 1]","[5, 3, 3, 5, 5, 1]"
624,"you've got to be fucking joking, my room has l...","[3, 3, 1, 1, 2, 1]","[2, 4, 1, 2, 3, 2]","[

In [5]:
df = df.loc[:, ["tweet", "racism", "annotatorAge","annotatorRace", "annotatorGender"]]
df

,tweet,racism,annotatorAge,annotatorRace,annotatorGender
0,""" you ugly bitch!!!""- that seems to be my real...","[1, 1, 1, 4]","[25.0, 35.0, 50.0, 25.0]","[black, black, white, white]","[man, man, woman, woman]"
1,"""""""I am happy to report that drowning does kil...","[1, 1, 5, 5, 1, 1]","[40.0, 30.0, 40.0, 30.0, 25.0, 25.0]","[black, black, white, white, white, white]","[woman, woman, woman, man, man, man]"
2,"""""""Let them praise the name of the LORD: for h...","[1, 1, 5, 1, 1, 1]","[30.0, 40.0, 35.0, 40.0, 25.0, 45.0]","[black, black, white, white, white, white]","[woman, woman, man, woman, man, woman]"
3,"""""""That was decades ago, Cecil! Anyway, we wan...","[1, 1, 5, 5, 1, 1]","[35.0, 40.0, 35.0, 40.0, 35.0, 45.0]","[black, black, white, white, white, white]","[man, woman, man, woman, woman, woman]"
4,"""""""This Jestxh idiot keeps riding me in dms, t...","[1, 3, 5, 1, 1]","[45.0, 45.0, 45.0, 60.0, 40.0]","[black, black, white, white, white]","[man, man, woman, woman, woman]"
...,...,...,...,...,...
621,women are not dumb bitches,"[1, 5, 1, 1]","[35.0, 30.0, 45.0, 65.0]","[black, black, white, white]","[man, woman, man, woman]"
622,worst part about getting highlights is sitting...,"[1, 5, 2, 3, 2, 1]","[45.0, 45.0, 35.0, 35.0, 25.0, 50.0]","[black, black, white, white, white, white]","[man, man, woman, man, man, man]"
623,yids are not as honest as the rest of us,"[1, 1, 4, 1, 2, 1]","[30.0, 30.0, 40.0, 60.0, 35.0, 40.0]","[black, black, white, white, white, white]","[man, man, man, woman, woman, woman]"
624,"you've got to be fucking joking, my room has l...","[1, 4, 1, 3, 3, 1]","[40.0, 25.0, 40.0, 35.0, 45.0, 35.0]","[black, black, white, white, white, white]","[woman, woman, woman, woman, woman, woman]"


In [6]:
def extract_unique_values(df, columns):
    """
    Extract unique values from lists in the specified columns of a pandas DataFrame.
    
    Args:
        df (pd.DataFrame): The input DataFrame.
        columns (list): List of column names to extract unique values from.
    
    Returns:
        dict: A dictionary where keys are column names and values are sets of unique values.
    """
    unique_values = {}
    for col in columns:
        unique_values[col] = set(value for sublist in df[col] for value in sublist)
    return unique_values

extract_unique_values(df, ["annotatorAge", "annotatorRace", "annotatorGender"])

{'annotatorAge': {nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  18.0,
  nan,
  nan,
  25.0,
  nan,
  nan,
  30.0,
  nan,
  35.0,
  nan,
  40.0,
  nan,
  nan,
  45.0,
  nan,
  nan,
  50.0,
  nan,
  55.0,
  nan,
  nan,
  60.0,
  nan,
  65.0,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan},
 'annotatorRace': {'black',
  'hisp',
  'middleEastern',
  'na',
  'native',
  'other',
  'white'},
 'annotatorGender': {'man', 'na', 'nonBinary', 'woman'}}

Clear out NaNs from dataset

In [7]:
nan_age = df.annotatorAge.apply(lambda x: np.any(np.isnan(x)))
nan_race = df.annotatorRace.apply(lambda x: "na" in x)
nan_gender = df.annotatorGender.apply(lambda x: "na" in x)

nan_age.sum(), nan_race.sum(), nan_gender.sum()


(41, 41, 39)

In [8]:
def find_inconsistent_rows(df):
    inconsistent_rows = []
    
    for index, row in df.iterrows():
        list_lengths = [len(row[col]) for col in df.columns if isinstance(row[col], list)]
        
        if len(set(list_lengths)) > 1:  # Check if lengths are inconsistent
            inconsistent_rows.append(index)
    
    return df.loc[inconsistent_rows]


find_inconsistent_rows(df)

,tweet,racism,annotatorAge,annotatorRace,annotatorGender


### Results

In [9]:
def extract_annotations_and_attributes(
    df: pd.DataFrame, annot_col: str, attribute_col: str, key_col: str
) -> tuple[list, list]:
    all_annotations = []
    all_attributes = []
    all_keys = []

    for _, row in df.iterrows():
        all_annotations.extend(row[annot_col])
        all_attributes.extend(row[attribute_col])
        # extend the key for each value in the above extracted list
        all_keys.extend([row[key_col]] * len(row[annot_col]))

    return all_annotations, all_attributes, all_keys


def run_aposteriori(df, column):
    annotations, attributes, keys = extract_annotations_and_attributes(
        df=df, annot_col="racism", attribute_col=column, key_col="tweet"
    )
    stat = aposteriori.aposteriori_unimodality(
        annotations=annotations, factor_group=attributes, comment_group=keys
    )
    return pd.Series(stat)

In [10]:
print(run_aposteriori(df[~nan_age], "annotatorAge"))
print(run_aposteriori(df[~nan_gender], "annotatorGender"))
print(run_aposteriori(df[~nan_race], "annotatorRace"))

25.0    2.227540e-33
35.0    8.056433e-03
50.0    1.517690e-31
18.0    1.405492e-01
30.0    2.653637e-06
40.0    1.027556e-20
45.0    1.280582e-11
55.0    3.393591e-02
60.0    2.622263e-08
65.0    1.172850e-12
dtype: float64
man          1.000000e+00
woman        1.826425e-04
nonBinary    6.034729e-08
dtype: float64
black            9.710141e-01
white            1.000000e+00
hisp             5.379128e-10
middleEastern             NaN
other            1.726807e-03
dtype: float64


/home/dimits/anaconda3/envs/std/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1081: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/home/dimits/anaconda3/envs/std/lib/python3.11/site-packages/scipy/stats/_stats_py.py:1081: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


## Synthetic data

In [11]:
syn_df = pd.read_csv("data/synthetic.csv")
syn_df

,conv_id,message,intent,Toxicity,Argument Quality,age_user,sex_user,sexual_orientation_user,demographic_group_user,current_employment_user,education_level_user,age_annot,sex_annot,sexual_orientation_annot,demographic_group_annot,current_employment_annot,education_level_annot
0,028f869d-cf12-444a-aefc-62f8b6294acd,@FitnessJunkie07 Respect and honesty are unive...,"['Neutral', 'Neutral', 'Neutral', 'Neutral', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]","['25', '25', '25', '25', '25', '25', '25', '25...","['male', 'male', 'male', 'male', 'male', 'male...","['Bisexual', 'Bisexual', 'Bisexual', 'Bisexual...","['Asian', 'Asian', 'Asian', 'Asian', 'Asian', ...","['Software Developer', 'Software Developer', '...","[""Bachelor's"", ""Bachelor's"", ""Bachelor's"", ""Ba...","['28', '40', '32', '60', '65', '21', '50', '27...","['male', 'female', 'non-binary', 'male', 'male...","['Homosexual', 'Heterosexual', 'Bisexual', 'As...","['Caucasian', 'Pacific Islander', 'Black', 'Ca...","['Research Scientist', 'Travel Blogger', 'Cybe...","[""Bachelor's"", 'Some College', ""Master's"", 'Ph..."
1,028f869d-cf12-444a-aefc-62f8b6294acd,"@FitnessJunkie07, I agree, but sometimes chang...","['Neutral', 'Neutral', 'Neutral', 'Neutral', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]","['27', '27', '27', '27', '27', '27', '27', '27...","['female', 'female', 'female', 'female', 'fema...","['Lesbian', 'Lesbian', 'Lesbian', 'Lesbian', '...","['Arab', 'Arab', 'Arab', 'Arab', 'Arab', 'Arab...","['Poet', 'Poet', 'Poet', 'Poet', 'Poet', 'Poet...","[""Master's"", ""Master's"", ""Master's"", ""Master's...","['28', '40', '32', '60', '65', '21', '50', '27...","['male', 'female', 'non-binary', 'male', 'male...","['Homosexual', 'Heterosexual', 'Bisexual', 'As...","['Caucasian', 'Pacific Islander', 'Black', 'Ca...","['Research Scientist', 'Travel Blogger', 'Cybe...","[""Bachelor's"", 'Some College', ""Master's"", 'Ph..."
2,028f869d-cf12-444a-aefc-62f8b6294acd,"@FitnessJunkie07, I completely agree that resp...","['Community-oriented', 'Community-oriented', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3]","['58', '58', '58', '58', '58', '58', '58', '58...","['female', 'female', 'female', 'female', 'fema...","['Heterosexual', 'Heterosexual', 'Heterosexual...","['Hispanic', 'Hispanic', 'Hispanic', 'Hispanic...","['Nonprofit Organizer', 'Nonprofit Organizer',...","[""Bachelor's"", ""Bachelor's"", ""Bachelor's"", ""Ba...","['28', '40', '32', '60', '65', '21', '50', '27...","['male', 'female', 'non-binary', 'male', 'male...","['Homosexual', 'Heterosexual', 'Bisexual', 'As...","['Caucasian', 'Pacific Islander', 'Black', 'Ca...","['Research Scientist', 'Travel Blogger', 'Cybe...","[""Bachelor's"", 'Some College', ""Master's"", 'Ph..."
3,028f869d-cf12-444a-aefc-62f8b6294acd,"@FitnessJunkie07, I understand where you're co...","['Community-oriented', 'Community-oriented', '...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]","['58', '58', '58', '58', '58', '58', '58', '58...","['female', 'female', 'female', 'female', 'fema...","['Heterosexual', 'Heterosexual', 'Heterosexual...","['Hispanic', 'Hispanic', 'Hispanic', 'Hispanic...","['Nonprofit Organizer', 'Nonprofit Organizer',...","[""Bachelor's"", ""Bachelor's"", ""Bachelor's"", ""Ba...","['28', '40', '32', '60', '65', '21', '50', '27...","['male', 'female', 'non-binary', 'male', 'male...","['Homosexual', 'Heterosexual', 'Bisexual', 'As...","['Caucasian', 'Pacific Islander', 'Black', 'Ca...","['Research Scientist', 'Travel Blogger', 'Cybe...","[""Bachelor's"", 'Some College', ""Master's"", 'Ph..."
4,028f869d-cf12-444a-aefc-62f8b6294acd,@LoyalVolunteer54 @FitnessJunkie07 It's import...,"['Moderator', 'Moderator', 'Moderator', 'Moder...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]","[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, na